# Chat with U
> seq2seq 모델과 Tensorflow로 제작한 Chatbot 튜토리얼입니다.

### Dataset Download
Chatbot 학습에 필요한 Dataset을 불러옵니다.
- [songys/Chatbot_data](https://github.com/songys/Chatbot_data.git)
- 문답 페어 11,876개
- `Q`: 질문
- `A`: 답변

In [9]:
!git clone https://github.com/songys/Chatbot_data.git

fatal: destination path 'Chatbot_data' already exists and is not an empty directory.


In [10]:
import pandas as pd
corpus = pd.read_csv('/content/Chatbot_data/ChatbotData.csv')

In [11]:
corpus.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [12]:
corpus.Q.head()

0             12시 땡!
1        1지망 학교 떨어졌어
2       3박4일 놀러가고 싶다
3    3박4일 정도 놀러가고 싶다
4            PPL 심하네
Name: Q, dtype: object

In [13]:
corpus.A.head()

0     하루가 또 가네요.
1      위로해 드립니다.
2    여행은 언제나 좋죠.
3    여행은 언제나 좋죠.
4     눈살이 찌푸려지죠.
Name: A, dtype: object

In [14]:
# dataset을 list 형태로 변환
q_list = []
a_list = []

for q, a in zip(corpus.Q, corpus.A):
    q_list.append(q)
    a_list.append(a)

In [15]:
# RAM 용량 제한으로 인한 데이터 개수 줄이기
q_list = q_list[:1000]
a_list = a_list[:1000]

In [16]:
q_list[:5]

['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']

### Preprocess
형태소 분석
- Konlpy의 Okt 분석기를 사용합니다.
   

토큰 추가
- `SOS`: Start Of Sentence
- `EOS`: End Of Sentence

In [17]:
!pip install konlpy

In [18]:
from konlpy.tag import Okt
okt = Okt()

In [19]:
sentence = "오늘은 먹고싶은게 딱히 없지만, 딱새우는 먹고싶어요"
okt.morphs(sentence)

['오늘', '은', '먹고', '싶은게', '딱히', '없지만', ',', '딱', '새우는', '먹고싶어요']

In [20]:
# 형태소 분석으로 분할된 단어들을 공백 기준으로 분리
def process_morph(sentence):
    return ' '.join(okt.morphs(sentence))

In [21]:
# 질문과 답변을 분리해서 처리
def morph_and_token(sentence, is_question=True):
    sentence = process_morph(sentence)
    if is_question:
        return sentence
    else:
        return ('<SOS> ' + sentence, sentence + ' <EOS>')

In [22]:
def preprocess(q_list, a_list):
    questions = []
    answer_input = []
    answer_output = []

    for q in q_list:
        question = morph_and_token(q, is_question=True)
        questions.append(question)

    for a in a_list:
        input_, output_ = morph_and_token(a, is_question=False)
        answer_input.append(input_)
        answer_output.append(output_)

    return questions, answer_input, answer_output

### Dataset Split
Encoder, Decoder의 관점으로 Dataset을 재구성합니다.
- `questions`: Encoder input  
- `answer_input`: Decoder input  
- `answer_output`: Decoder output

In [23]:
questions, answer_input, answer_output = preprocess(q_list, a_list)

In [24]:
questions[:5]

['12시 땡 !', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다', '3 박 4일 정도 놀러 가고 싶다', 'PPL 심하네']

In [25]:
answer_input[:5]

['<SOS> 하루 가 또 가네요 .',
 '<SOS> 위로 해 드립니다 .',
 '<SOS> 여행 은 언제나 좋죠 .',
 '<SOS> 여행 은 언제나 좋죠 .',
 '<SOS> 눈살 이 찌푸려지죠 .']

In [26]:
answer_output[:5]

['하루 가 또 가네요 . <EOS>',
 '위로 해 드립니다 . <EOS>',
 '여행 은 언제나 좋죠 . <EOS>',
 '여행 은 언제나 좋죠 . <EOS>',
 '눈살 이 찌푸려지죠 . <EOS>']

In [27]:
# vocab 제작에 사용
all_sentences = questions + answer_input + answer_output
all_sentences

['12시 땡 !',
 '1 지망 학교 떨어졌어',
 '3 박 4일 놀러 가고 싶다',
 '3 박 4일 정도 놀러 가고 싶다',
 'PPL 심하네',
 'SD 카드 망가졌어',
 'SD 카드 안 돼',
 'SNS 맞팔 왜 안 하지 ㅠㅠ',
 'SNS 시간 낭비 인 거 아는데 매일 하는 중',
 'SNS 시간 낭비 인데 자꾸 보게 됨',
 'SNS 보면 나 만 빼고 다 행복 해보여',
 '가끔 궁금해',
 '가끔 뭐 하는지 궁금해',
 '가끔 은 혼자 인게 좋다',
 '가난한 자의 설움',
 '가만 있어도 땀 난다',
 '가상 화폐 쫄딱 망함',
 '가스 불 켜고 나갔어',
 '가스 불 켜놓고 나온거 같아',
 '가스 비 너무 많이 나왔다 .',
 '가스 비 비싼데 감기 걸리겠어',
 '가스 비 장난 아님',
 '가장 확실한 건 뭘 까 ?',
 '가족 여행 가기 로 했어',
 '가족 여행 고고',
 '가족 여행 어디 로 가지 ?',
 '가족 있어 ?',
 '가족 관계 알려 줘',
 '가족 끼리 여행 간다 .',
 '가족 들 보고 싶어',
 '가족 들 이랑 서먹해',
 '가족 들 이랑 서먹해졌어',
 '가족 들 이랑 어디 가지 ?',
 '가족 들 이랑 여행 갈거야',
 '가족 여행 가야 지',
 '가족 이 누구 야 ?',
 '가족 이랑 여행 가려고',
 '가족 한테 스트레스 풀었어',
 '가출 할까 ?',
 '가출 해도 갈 데 가 없어',
 '간만 에 떨리니까 좋더라',
 '간만 에 쇼핑 중',
 '간만 에 휴식 중',
 '간식 뭐 먹을까',
 '간식 추천',
 '간장 치킨 시켜야지',
 '간접흡연 싫어',
 '갈까 말까 고민 돼',
 '갈까 말까 ?',
 '감 말랭이 먹고 싶다 .',
 '감 말랭이 먹어야지',
 '감기 같 애',
 '감기 걸린 것 같아',
 '감기 기운 이 있어',
 '감기 들 거 같 애',
 '감기 가 오려나',
 '감기 약 이 없어',
 '감기 인거 같 애',
 '감미로운 목소리 좋아',
 '감정 이 쓰레기통 처럼 엉망 진창 이야',
 '감

### Tokenization
- Vocab을 만들어줍니다.
- Text를 Sequence로 Encoding합니다.
- Padding으로 문장의 길이를 일정하게 맞춰줍니다.

In [28]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [29]:
# 토큰의 옵션 정의
# OOV는 Out Of Vocabulary
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')

In [30]:
# internal vocabulary 생성
tokenizer.fit_on_texts(all_sentences)

In [31]:
VOCAB_SIZE = len(tokenizer.word_index) + 1

In [32]:
VOCAB_SIZE

2311

In [33]:
print(tokenizer.word_index)

{'<OOV>': 1, '.': 2, '<SOS>': 3, '<EOS>': 4, '이': 5, '거': 6, '을': 7, '가': 8, '나': 9, '예요': 10, '사람': 11, '요': 12, '도': 13, '해보세요': 14, '잘': 15, '에': 16, '?': 17, '를': 18, '내': 19, '는': 20, '은': 21, '너무': 22, '저': 23, '하세요': 24, '마세요': 25, '보세요': 26, '수': 27, '하고': 28, '더': 29, '!': 30, '것': 31, '생각': 32, '봐요': 33, '안': 34, '할': 35, '이에요': 36, '하는': 37, '게': 38, '해': 39, '있을': 40, '많이': 41, '같아': 42, '마음': 43, '의': 44, '때': 45, '말': 46, '있어요': 47, '좋은': 48, '시간': 49, '못': 50, '만': 51, '들': 52, '친구': 53, '하지': 54, '자신': 55, '같아요': 56, '한': 57, '인': 58, '적': 59, '공부': 60, '하면': 61, '제': 62, '좀': 63, '해주세요': 64, '으로': 65, '싶어': 66, '네': 67, '다': 68, '남자친구': 69, '고': 70, '일': 71, '같이': 72, '있는': 73, '선물': 74, '죠': 75, '그': 76, '내일': 77, '다른': 78, '지금': 79, '이야기': 80, '고민': 81, '준비': 82, '보다': 83, '바라요': 84, '드세요': 85, '이네': 86, '로': 87, '있어': 88, '좋죠': 89, '가보세요': 90, '당신': 91, '될': 92, '왜': 93, '가족': 94, '한테': 95, '그런': 96, '곳': 97, '너': 98, '있으면': 99, '랑': 100, '거짓말': 101, '에는': 102, '신경'

In [34]:
# vocab 확인해보기
for word, index in tokenizer.word_index.items():
    print(f'{index}\t\t\t{word}')
    if index == VOCAB_SIZE:
        break

1			<OOV>
2			.
3			<SOS>
4			<EOS>
5			이
6			거
7			을
8			가
9			나
10			예요
11			사람
12			요
13			도
14			해보세요
15			잘
16			에
17			?
18			를
19			내
20			는
21			은
22			너무
23			저
24			하세요
25			마세요
26			보세요
27			수
28			하고
29			더
30			!
31			것
32			생각
33			봐요
34			안
35			할
36			이에요
37			하는
38			게
39			해
40			있을
41			많이
42			같아
43			마음
44			의
45			때
46			말
47			있어요
48			좋은
49			시간
50			못
51			만
52			들
53			친구
54			하지
55			자신
56			같아요
57			한
58			인
59			적
60			공부
61			하면
62			제
63			좀
64			해주세요
65			으로
66			싶어
67			네
68			다
69			남자친구
70			고
71			일
72			같이
73			있는
74			선물
75			죠
76			그
77			내일
78			다른
79			지금
80			이야기
81			고민
82			준비
83			보다
84			바라요
85			드세요
86			이네
87			로
88			있어
89			좋죠
90			가보세요
91			당신
92			될
93			왜
94			가족
95			한테
96			그런
97			곳
98			너
99			있으면
100			랑
101			거짓말
102			에는
103			신경
104			좋을
105			주세요
106			날씨
107			돼
108			혼자
109			여행
110			결정
111			기분
112			,
113			가세
114			있죠
115			애
116			게임
117			살
118			봐
119			누구
120			걸
121			과
122			없어요
123			가장
124			없어
125			남
126			남편

In [35]:
# Text to Sequence Encoding
questions_sequence = tokenizer.texts_to_sequences(questions)
answer_input_sequence = tokenizer.texts_to_sequences(answer_input)
answer_output_sequence = tokenizer.texts_to_sequences(answer_output)

In [36]:
tokenizer.word_index['곱창']

1117

In [37]:
questions[:5]

['12시 땡 !', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다', '3 박 4일 정도 놀러 가고 싶다', 'PPL 심하네']

In [38]:
questions_sequence[:5]

[[1616, 1617, 30],
 [957, 1618, 1619, 1057],
 [554, 1058, 1059, 958, 243, 155],
 [554, 1058, 1059, 1060, 958, 243, 155],
 [1620, 1621]]

In [39]:
# Padding Hyperparameter
MAX_LENGTH = 30

In [40]:
# post -> 문장을 잘라낼때 뒷부분부터 잘라주고, Padding을 해줄때 뒷부분부터 채워넣음
questions_padded = pad_sequences(questions_sequence, maxlen=MAX_LENGTH, padding='post', truncating='post')
answer_input_padded = pad_sequences(answer_input_sequence, maxlen=MAX_LENGTH, padding='post', truncating='post')
answer_output_padded = pad_sequences(answer_output_sequence, maxlen=MAX_LENGTH, padding='post', truncating='post')

In [41]:
questions_padded[:5]

array([[1616, 1617,   30,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [ 957, 1618, 1619, 1057,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [ 554, 1058, 1059,  958,  243,  155,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [ 554, 1058, 1059, 1060,  958,  243,  155,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0],
       [1620, 1621,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0, 

In [42]:
questions_padded.shape

(1000, 30)

### Vectorization
- 각 단어들을 One-Hot Encoding 변환
- Vocab의 index를 참조해 다시 text 형태로 변환 (예측 과정에서 호출)

In [43]:
#One-Hot Encoding
def convert_to_one_hot(padded):
    one_hot_vector = np.zeros((len(padded), MAX_LENGTH, VOCAB_SIZE))

    for i, sequence in enumerate(padded):
        for j, index in enumerate(sequence):
            one_hot_vector[i, j, index] = 1
    
    return one_hot_vector    

In [44]:
#keras의 to_categorical을 쓸려면 뒤의 코드 중 Seq2Seq 함수에서 Vocab size를 answer_output의 vocab 개수랑 맞춰줘야됨
#answer_output_one_hot = to_categorical(answer_output_padded)

In [45]:
answer_input_one_hot = convert_to_one_hot(answer_input_padded)
answer_output_one_hot = convert_to_one_hot(answer_output_padded)

In [46]:
answer_output_one_hot.shape

(1000, 30, 2311)

In [47]:
len(answer_output_one_hot[0][0])

2311

In [48]:
# 예측 값을 단어사전에서 찾아와 문자열로 변환
def index_to_text(indexs, end_token):
    sentence = ' '

    for i in indexs:
        if i == end_token:
            break;

        if i > 0 and tokenizer.index_word[i] is not None:
            sentence += tokenizer.index_word[i]
        else:
            sentence += ''

        sentence += ' '
    return sentence

### Generate Model
- Encoder 정의
- Decoder 정의

In [49]:
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Model

In [50]:
class Encoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim, input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units, return_state=True)

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.dropout(x)
        x, hidden_state, cell_state = self.lstm(x)
        # return context vector
        return [hidden_state, cell_state]

In [51]:
class Decoder(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim, input_length=time_steps)
        self.dropout = Dropout(0.2)
        self.lstm = LSTM(units, return_state=True, return_sequences=True, )
        self.dense = Dense(vocab_size, activation='softmax')

    def call(self, inputs, initial_state):
        x = self.embedding(inputs)
        x = self.dropout(x)
        # x는 Decoder의 input값을 의미
        # Encoder의 return값(context vector)이 Decoder의 initial_state
        x, hidden_state, cell_state = self.lstm(x, initial_state=initial_state)
        x = self.dense(x)
        return x, hidden_state, cell_state

In [52]:
class Seq2Seq(tf.keras.Model):
    def __init__(self, units, vocab_size, embedding_dim, time_steps, start_token, end_token):
        super(Seq2Seq, self).__init__()
        self.start_token = start_token
        self.end_token = end_token
        self.time_steps = time_steps

        self.encoder = Encoder(units, vocab_size, embedding_dim, time_steps)
        self.decoder = Decoder(units, vocab_size, embedding_dim, time_steps)

    def call(self, inputs, training=True):
        # 학습 상태의 경우
        if training:
            encoder_inputs, decoder_inputs = inputs
            context_vector = self.encoder(encoder_inputs)
            decoder_outputs, _, _ = self.decoder(inputs=decoder_inputs, initial_state=context_vector)
            return decoder_outputs

        # 예측 상태의 경우
        else:
            context_vector = self.encoder(inputs)
            target_seq = tf.constant([[self.start_token]], dtype=tf.float32)
            results = tf.TensorArray(tf.int32, self.time_steps)

            for i in tf.range(self.time_steps):
                decoder_outputs, decoder_hidden, decoder_cell = self.decoder(target_seq, initial_state=context_vector)
                decoder_outputs = tf.cast(tf.argmax(decoder_outputs, axis=-1), dtype=tf.int32)
                decoder_outputs = tf.reshape(decoder_outputs, shape=(1, 1))
                results = results.write(i, decoder_outputs)

                if decoder_outputs == self.end_token:
                    break;

                target_seq = decoder_outputs
                context_vector = [decoder_hidden, decoder_cell]

            return tf.reshape(results.stack(), shape=(1, self.time_steps))

### Train

In [53]:
BUFFER_SIZE = 1000
BATCH_SIZE = 16
EMBEDDING_DIM = 100
TIME_STEPS = MAX_LENGTH

START_TOKEN = tokenizer.word_index['<SOS>']
END_TOKEN = tokenizer.word_index['<EOS>']

# LSTM에 들어가는 UNITS
UNITS = 128

VOCAB_SIZE = len(tokenizer.word_index) + 1
DATA_LENGTH = len(questions)
SAMPLE_SIZE = 3
NUM_EPOCHS = 20

In [54]:
# 모델 생성 및 컴파일
seq2seq = Seq2Seq(UNITS, VOCAB_SIZE, EMBEDDING_DIM, TIME_STEPS, START_TOKEN, END_TOKEN)
seq2seq.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [55]:
# 예측
def make_prediction(model, question_inputs):
    results = model(inputs=question_inputs, training=False)
    results = np.asarray(results).reshape(-1)
    return results

In [56]:
# 주어진 epoch만큼 모델 학습
for epoch in range(NUM_EPOCHS):
    print(f'processing epoch: {epoch * 10 + 1}...')
    seq2seq.fit([questions_padded, answer_input_padded],
                answer_output_one_hot,
                epochs=10,
                batch_size=BATCH_SIZE,
                )
    
    samples = np.random.randint(DATA_LENGTH, size=SAMPLE_SIZE)

    for idx in samples:
        question_inputs = questions_padded[idx]
        results = make_prediction(seq2seq, np.expand_dims(question_inputs, 0))
        results = index_to_text(results, END_TOKEN)

        print(f'Q: {questions[idx]}')
        print(f'A: {results}\n')
        print()

processing epoch: 1...
Epoch 1/10
63/63 [==============================] - 4s 10ms/step - loss: 3.3794 - acc: 0.7417
Epoch 2/10
63/63 [==============================] - 1s 10ms/step - loss: 1.4015 - acc: 0.7714
Epoch 3/10
63/63 [==============================] - 1s 10ms/step - loss: 1.2947 - acc: 0.7853
Epoch 4/10
63/63 [==============================] - 1s 10ms/step - loss: 1.2026 - acc: 0.8153
Epoch 5/10
63/63 [==============================] - 1s 10ms/step - loss: 1.1476 - acc: 0.8280
Epoch 6/10
63/63 [==============================] - 1s 10ms/step - loss: 1.1086 - acc: 0.8295
Epoch 7/10
63/63 [==============================] - 1s 10ms/step - loss: 1.0779 - acc: 0.8303
Epoch 8/10
63/63 [==============================] - 1s 10ms/step - loss: 1.0523 - acc: 0.8319
Epoch 9/10
63/63 [==============================] - 1s 14ms/step - loss: 1.0297 - acc: 0.8328
Epoch 10/10
63/63 [==============================] - 1s 10ms/step - loss: 1.0108 - acc: 0.8347
Q: 나 속 은 듯
A:  저 을 이 이 이 이 예요 . 


Q

### Prediction
- 사용자로부터 입력받은 문장의 전처리를 해줍니다.
- 전처리 한 문장을 입력해 예측값을 얻습니다.

In [57]:
# 입력받은 문장을 전처리
def make_question(sentence):
    sentence = morph_and_token(sentence)
    question_sequence = tokenizer.texts_to_sequences([sentence])
    question_padded = pad_sequences(question_sequence, maxlen=MAX_LENGTH, truncating='post', padding='post')
    return question_padded

In [58]:
make_question('1지망 학교 떨어졌어')

array([[ 957, 1618, 1619, 1057,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)

In [59]:
# make_question으로 전처리 후 make_prediction으로 예측
def run_chatbot(question):
    question_inputs = make_question(question)
    results = make_prediction(seq2seq, question_inputs)
    results = index_to_text(results, END_TOKEN)
    return results

### Test
- 챗봇과 대화를 이어갈 수 있는 코드
- 질문이 questions.csv 파일에 기록됩니다.

In [60]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [61]:
EXIT = "대화종료"
input_values = []

while True:
    user_input = input('\nQuestion: ')
    if user_input == EXIT:
        break
    input_values.append(user_input)
    answer = run_chatbot(user_input)
    print(f'Answer: {answer}')
    print('---------------------------')


Question: 나 배고파
Answer:  저 는 위로 해드리는 로봇 이에요 . 
---------------------------

Question: 곱창 먹고싶어
Answer:  현실 을 꿈 처럼 만들어 봐요 . 
---------------------------

Question: 볶음밥도 먹고싶다
Answer:  저 는 위로 해드리는 로봇 이에요 . 
---------------------------

Question: 나 졸려ㅠㅠ
Answer:  저 도 궁금하네요 . 
---------------------------

Question: 나 졸려
Answer:  자신 을 위 한 결정 을 내리길 바라요 . 
---------------------------

Question: 그럼 지금 자러갈게
Answer:  저 는 위로 해드리는 로봇 이에요 . 
---------------------------

Question: 대화종료


In [62]:
input_values

['나 배고파', '곱창 먹고싶어', '볶음밥도 먹고싶다', '나 졸려ㅠㅠ', '나 졸려', '그럼 지금 자러갈게']

In [63]:
#import csv

#file_path = '/content/gdrive/MyDrive/AI_Project/ChatBot/questions.csv'
#with open(file_path, 'w', newline='') as questions:
#    write = csv.writer(questions)
#    write.writerow(input_values)

### One-hot Encoding test

In [64]:
texts = ['오늘 점심은 떡볶이','오늘 저녁은 치킨','야식은 햄버거']

In [65]:
t = Tokenizer(filters='')
t.fit_on_texts(texts)
print(t.word_index)
print(len(t.word_index) + 1)

{'오늘': 1, '점심은': 2, '떡볶이': 3, '저녁은': 4, '치킨': 5, '야식은': 6, '햄버거': 7}
8


In [66]:
sequences = t.texts_to_sequences(texts)
sequences

[[1, 2, 3], [1, 4, 5], [6, 7]]

In [67]:
sequences = pad_sequences(sequences, maxlen=5, padding='post', truncating='post')
print(sequences)
print(sequences.shape)

[[1 2 3 0 0]
 [1 4 5 0 0]
 [6 7 0 0 0]]
(3, 5)


In [68]:
one_hot = to_categorical(sequences)
print(one_hot)
print(one_hot.shape)

[[[0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]]]
(3, 5, 8)


In [69]:
print(one_hot[0])
print(one_hot[0].shape)

[[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]]
(5, 8)
